### Espaços Vetoriais

É um modelo que é útil para fazer distinção do significado das palavras. Isto é, mesmo com palavras iguais em contexto as quais possuem significados distintos ou vice-versa, esse modelo se aplica como uma solução útil para tais situações.

Isto indica também, que espaços vetoriais será possível capturar informações de relação entre palavras.
Informações como:

- Quem
- O que
- Como
- Porque

serão destiguíveis

In [25]:
import numpy as np
import pandas as pd

corpo = [
    "Eu gosto de comer biscoito",
    "Eu odeio você",
    "Eu te amo!!",
    "Você é o amor da minha vida",
    "Vá para a baixa da égua",
    "Você é uma pessoa repugnante",
    "Você é desagradável",
    "Você é amável"
]

classes = [
    1,
    0,
    1,
    1,
    0,
    0,
    0,
    1
]

vocabulario = {}
indices = {}
pontuacao = ['!',","]
palavrasPausa = ["a", "é", "uma", "de", "o", "da"]
def criarVocabulario(corpo):
    v = {}
    i = {}
    index = 0
    for frase in corpo:
        for palavra in frase.split(" "):
            for p in pontuacao:
                palavra = palavra.replace(p,'').lower()
            if palavra not in list(v.keys()) and palavra not in palavrasPausa:
                v[palavra] = index
                i[index] = palavra
                index+=1
    return v,i    

vocabulario, indices = criarVocabulario(corpo)
vocabulario

{'eu': 0,
 'gosto': 1,
 'comer': 2,
 'biscoito': 3,
 'odeio': 4,
 'você': 5,
 'te': 6,
 'amo': 7,
 'amor': 8,
 'minha': 9,
 'vida': 10,
 'vá': 11,
 'para': 12,
 'baixa': 13,
 'égua': 14,
 'pessoa': 15,
 'repugnante': 16,
 'desagradável': 17,
 'amável': 18}

### Matriz de co-ocorrência

Os vetores que representam as palavras do vocabulário podem estar localizados em uma chamada matriz de co-ocorrência. Tal matriz, pode ser composta de duas formas:

#### Palavra por palavra

Uma matriz de co-ocorrência feita palavra por palavra, procura computar a contagem de palavras vizinhas em um vetor de tamanho N para N sendo o total de palavras no vocabulário. Uma palavra é vizinha de outra se e somente se a distância entre ambas é inferior a K (hiperparâmetro). 

#### Palavra por documento

É possível também contar a quantidade de palavras existentes em uma série de documentos com contextos distintos. Assim sendo, o vetor é representado por um tamanho M, onde M é o total de assuntos (classes) dos documentos.

In [35]:
matrizCoocorrencia = np.zeros((len(list(vocabulario.keys())),(len(list(vocabulario.keys())))))

def tokenizar(frase):
    tokens = []
    for palavra in frase.split(" "):
        for p in pontuacao:
            palavra = palavra.replace(p,'').lower()
        if palavra not in palavrasPausa:
            tokens.append(palavra)
    return tokens

def contarFrequencia(fraseTokens,k = 2):
    for index in range(len(fraseTokens)):
        for auxIndex in range(index + 1,len(fraseTokens)):
            if(abs(index - auxIndex) <= k):
                matrizCoocorrencia[vocabulario[fraseTokens[index]],vocabulario[fraseTokens[auxIndex]]] += 1
                matrizCoocorrencia[vocabulario[fraseTokens[auxIndex]],vocabulario[fraseTokens[index]]] += 1

for frase in corpo:
    contarFrequencia(tokenizar(frase),k=2)
    
matrizCoocorrencia.shape

(19, 19)

In [30]:
def normalize(vec):
    return vec/(np.linalg.norm(vec))
                
def euclid(vac,vec):
    return np.linalg.norm(vec - vac)

a,b= 0,1

np.dot(normalize(matrizCoocorrencia[a]),normalize(matrizCoocorrencia[b])),euclid(normalize(matrizCoocorrencia[a]),normalize(matrizCoocorrencia[b]))

(0.2357022603955159, 1.2363638134501385)

A distância euclidiana, apesar de ajudar a identificar a similaridade entre vetor não é a mais utilizada. A similaridade por cosseno é mais segura pois não se baseia no tamanho do corpo utilizado.

A formula do cosseno de theta é dada por:

$$ cos \theta = \frac{<v,w>}{||v||*||w||}$$

Por outro lado, a distancia euclidiana pode ser calulada pela fórmula

$$ dist(v,w) = || v - w ||$$

Lembrando que $||v|| = \sqrt{\sum_{i}^{len(v)}{v[i]^2}}$

### Principal Component Analysis

Permite representar meus dados em dimenções menores, facilitando a vizualização dos mesmos.

Passo a passo:

1. Computar a matriz normalizada pela média
2. Computar matriz de covariância $\Sigma$
3. Aplicar SVD (single value decomposition) em $\Sigma$, obtendo $U S V$
4. A matriz reduzida é $US[:,:k]$ onde $k$ é a dimensão alvo

Isto funciona pois serão usados os $k$ autovetores que tem maiores autovalores. Isto significa que são as partes que mais retêm informação da matriz

In [31]:
#np.linalg.svd(matrizCoocorrencia)

matrizCoovariancia = (matrizCoocorrencia - np.mean(matrizCoocorrencia,axis=0))/np.std(matrizCoocorrencia,axis=0)
print(matrizCoovariancia.shape)
U,S,V = np.linalg.svd(matrizCoovariancia)

(19, 19)


In [32]:
U.shape, S.shape, V.shape

((19, 19), (19,), (19, 19))

### Redução de dimencionalidade (PCA)

Ao aplicar decomposição de valor singular na matriz de covariância, obtemos U com os auto-vetores de A e S com os seus autovalores ordenados.

O resultado do produto entre ambos irá gerar uma matriz onde as primeiras colunas possuem os valores mais significativos de A, pois afinal serão os produtos dos maiores autovalores pelos autovetores

In [33]:
index = 0
chaves = list(vocabulario.keys())
for line in (U * S)[:,:2]:
    print(chaves[index],"\t\t",line)
    index +=1

eu 		 [0.19759734 4.5848284 ]
gosto 		 [-0.50651114  1.80637789]
comer 		 [-0.50651114  1.80637789]
biscoito 		 [-0.82526318  1.81977248]
odeio 		 [-0.2817173   0.47342932]
você 		 [ 8.41621035 -1.57122539]
te 		 [-0.29896355  1.18001976]
amo 		 [-0.29896355  1.18001976]
amor 		 [ 0.274423   -0.33684374]
minha 		 [ 0.274423   -0.33684374]
vida 		 [ 1.03686773 -0.83242115]
vá 		 [-1.4983054  -1.75844895]
para 		 [-2.06337387 -3.04735723]
baixa 		 [-2.06337387 -3.04735723]
égua 		 [-1.4983054  -1.75844895]
pessoa 		 [ 0.44672364 -0.19110348]
repugnante 		 [ 0.44672364 -0.19110348]
desagradável 		 [-0.62584014  0.11016391]
amável 		 [-0.62584014  0.11016391]
